# メタデータから研究データ品質を検証する

- タイトル案
    - 【DMP＋α】に基づいた適切なデータ管理が行われているかを検証する
    - メタデータから研究データ品質を検証する

研究データに付与されているメタデータを検証し、研究データの品質をレポートします。  
検証結果のレポートは、この環境で確認したあとにGIN-forkへ同期するか破棄するか選択いただけます。  
GIN-forkへ同期する場合は、研究リポジトリの直下にありますvalidation_resultsフォルダでバージョン管理されます。

## 0. 研究リポジトリ名を確認する

以下のセルを実行すると、この実験実行環境で操作する研究リポジトリ名を確認できます。 

In [ ]:
import os
os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils
utils.show_name('green')

## 1. 研究リポジトリのメタデータを用意する

メタデータを検証するための準備として、研究リポジトリのメタデータの取得と整形を行います。

In [ ]:
import os
os.chdir(os.environ['HOME'])

# 現在のブランチ名を取得する
result = !git branch --contains
current_branch = result[0].replace('* ', '')

# repository_idを取得する
from WORKFLOWS.utils.repository_id import repository_id
repo_id = repository_id.get_repo_id()

# ginforkのtokenを取得する
from WORKFLOWS.utils.token import token
ginfork_token = token.get_ginfork_token()

# GIN-forkのメタデータ取得APIを実行する
## TBD ##

# PKGでメタデータをRO-Crate化する

## 2. メタデータ検証を検証する

メタデータの検証と検証結果の生成を行う検証サービスに、メタデータの検証を依頼します。

In [ ]:
import requests

try:
    req_response = requests.post("https://example.com/**/validate?entityIds=a&entityIds=b", json=crate_json)
    result = req_response.json
except:
    print('検証サービスに接続できません。')
else:
    if req_response.status_code == 200:
        request_id = result['request_id']
        print('正常に検証が依頼されました。次の処理にお進みください。\nリクエストID：' + request_id)
    elif req_response.status_code == 400:
        print('BadRequest')
        print(result['message'])
    elif req_response.status_code == 500:
        print('InternalServerError')
        print(result['message'])

## 3. 検証結果を確認する

検証サービスから検証結果を取得し、結果を出力します。

In [ ]:
import os
import json
import requests
import time
from IPython.display import display, HTML

counter = 5
while counter >0:
    try:
        result_response = requests.get("https://example.com/*/" + request_id)
        result = result_response.json
        counter -= 1
    except:
        print('検証サービスに接続できません。')
        break
    else:
        if result_response.status_code == 200:
            status = result['status']
            if status == 'UNKNOWN':
                print('リクエストID：' + request_id + 'は正常にリクエストできていません。')
                break
            elif any([status == 'QUEUED', status == 'RUNNING'])''
                print('リクエストID：' + request_id + 'はまだ検証完了していません。時間をおいて再度確認します。')
                # アスケイドさんの単体テストで5sec待っていたのを参考に5秒に設定している
                time.sleep(5)
                continue
            elif status == 'COMPLETE':
                os.chdir(os.environ['HOME'])
                # .gitignore対象の.tmpフォルダ配下に検証結果を格納する
                tmp_result_folder = '.tmp/' + request_id
                !mkdir -p tmp_result_folder
                tmp_files = [
                    [result['request']['roCrate'], tmp_result_folder +'/ro_crate.json'],
                    [result['request']['entityIds'], tmp_result_folder +'/entity_ids.json'],
                    [result['results'], tmp_result_folder + '/results.json']
                ]
                for file in tmp_files:
                    with open(file[1], 'w') as f:
                        json.dump(file[0], f, indent=4)
                print('正常に検証結果を取得しました。以下のリンクから検証結果を確認し、次にお進みください。')
                display(HTML('<a href=../../../../edit/' + tmp_result_folder + '/results.json' + '>検証結果を確認する</a>'))
                break
            elif any([status == 'FAILED', status == 'EXECUTOR_ERROR']):
                print('正常に検証できませんでした。')
                break
            elif status == 'CANCELING':
                print('リクエストID：' + request_id + 'は現在キャンセル中です。')
                break
            elif status == 'CANCELED':
                print('リクエストID：' + request_id + 'はキャンセルされました。')
                break

        elif result_response.status_code == 400:
            print('BadRequest')
            print(result['message'])
            break
        elif result_response.status_code == 500:
            print('InternalServerError')
            print(result['message'])
            break
                      
else:
    print('検証に時間がかかっています。時間をおいて再度このセルを実行してください。')

## 4. 研究リポジトリに同期する

このタスクの実行結果を研究リポジトリに同期します。  
検証結果を同期対象に含めるか含めないかは、「4.1. 検証結果を研究リポジトリに同期するか選択する」で選択できます。

### 4.1. 検証結果を研究リポジトリに同期するか選択する

In [ ]:
import panel as pn

pn.extension()
column = pn.Column()

def save_selection_result(event):
    global need_sync
    done_button.button_type = "success"
    done_button.name = "選択完了しました。次の処理にお進みください。"
    need_sync = True if select.value == 1 else False

select = pn.widgets.Select(name='検証結果を同期するかどうか選択した後、完了ボタンをクリックしてください。', options={'同期する': 1, '同期しない':2})
done_button = pn.widgets.Button(name= "選択を完了する", button_type= "primary")
done_button.on_click(save_selection_result)
column.append(select)
column.append(done_button)
column

### 4.2. 研究リポジトリに同期する

In [ ]:
import os
from IPython.display import display, Javascript

os.chdir('/home/jovyan/WORKFLOWS/FLOW/')
from util.scripts import utils

os.chdir(os.environ['HOME'])
need_sync_path = ['/home/jovyan/WORKFLOWS/FLOW/02_experimental_phase/base_validate_metadata.ipynb']
if need_sync == True:
    !mv -f $tmp_result_folder validation_results
    need_sync_path.append('validation_results')

display(Javascript('IPython.notebook.save_checkpoint();'))
utils.syncs_with_repo(git_path=need_sync_path, gitannex_path=None, gitannex_files=None, message='メタデータ検証')

## 5. 研究ワークフロー機能トップページに遷移する

以下のセルを実行し、表示されるリンクをクリックして研究ワークフロー機能トップページに戻ってください。

In [ ]:
from IPython.display import display, HTML, Javascript
display(HTML("<a href='../../base_FLOW.ipynb'>研究ワークフロー機能トップページに遷移する</a>"))
display(Javascript('IPython.notebook.save_checkpoint();'))